# Step 1: Install required libraries

In [ ]:
!pip install transformers torch nltk evaluate

# Step 2: Import libraries

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import nltk
from collections import Counter
import evaluate

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

# Step 3: Load the Qwen model and tokenizer

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Step 4: Define Shannon entropy calculation

In [ ]:
def shannon_entropy(probabilities):
    """
    Calculate Shannon entropy for a probability distribution.
    """
    return -np.sum(probabilities * np.log(probabilities + 1e-10))  # Add small epsilon to avoid log(0)

# Step 5: Implement entropy-controlled text generation

In [ ]:
def generate_text_with_entropy_control(prompt, max_length=50, target_entropy=1.0, temperature=1.0):
    """
    Generate text using entropy-controlled decoding with the Qwen model.

    Args:
        prompt (str): Input text or instruction to start generation.
        max_length (int): Maximum length of the generated text.
        target_entropy (float): Desired entropy level for diversity control.
        temperature (float): Sampling temperature (higher = more random).

    Returns:
        str: Generated text.
    """
    # Format the input as an instruction (optional, depending on the model)
    formatted_prompt = f"Instruction: {prompt}\nOutput:"
    input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt")
    generated_text = formatted_prompt

    for _ in range(max_length):
        # Get model outputs
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits[:, -1, :] / temperature  # Apply temperature scaling

        # Compute probabilities
        probs = torch.softmax(logits, dim=-1).squeeze().numpy()

        # Calculate entropy
        entropy = shannon_entropy(probs)

        # Adjust sampling based on target entropy
        if entropy < target_entropy:
            # Encourage diversity (higher entropy)
            probs = probs ** (1 / temperature)
            probs = probs / np.sum(probs)  # Renormalize
        else:
            # Encourage coherence (lower entropy)
            probs = probs ** temperature
            probs = probs / np.sum(probs)  # Renormalize

        # Sample the next token
        next_token_id = np.random.choice(len(probs), p=probs)
        next_token = tokenizer.decode([next_token_id])

        # Append the token to the generated text
        generated_text += next_token

        # Update input_ids for the next step
        input_ids = torch.cat([input_ids, torch.tensor([[next_token_id]])], dim=1)

    return generated_text

# Step 6: Define evaluation metrics

In [ ]:
# Perplexity: Measures how well the model predicts the text
def calculate_perplexity(text, model, tokenizer):
    """
    Calculate perplexity for a given text.

    Args:
        text (str): The generated text.
        model: The language model.
        tokenizer: The tokenizer.

    Returns:
        float: The perplexity score.
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Calculate perplexity
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss).item()

    return perplexity

In [19]:
def distinct_n(text, n=2):
    tokens = nltk.word_tokenize(text)
    n_grams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
    unique_n_grams = Counter(n_grams)
    return len(unique_n_grams) / len(n_grams) if len(n_grams) > 0 else 0

In [20]:
def repetition_rate(text, n=2):
    tokens = nltk.word_tokenize(text)
    n_grams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
    n_gram_counts = Counter(n_grams)
    repeated = sum(count > 1 for count in n_gram_counts.values())
    return repeated / len(n_gram_counts) if len(n_gram_counts) > 0 else 0

# Step 7: Generate text

In [ ]:
# Example prompt
prompt = "Write a short story about a robot learning to paint."

# Generate text with low entropy (more deterministic)
low_entropy_text = generate_text_with_entropy_control(prompt, max_length=100, target_entropy=0.5)

# Generate text with high entropy (more diverse)
high_entropy_text = generate_text_with_entropy_control(prompt, max_length=100, target_entropy=2.0)

# Clean the generated text

In [16]:
def clean_generated_text(generated_text):
    """
    Extract the text after "Output:" in the generated text.

    Args:
        generated_text (str): The full generated text including the prompt.

    Returns:
        str: The cleaned generated text.
    """
    # Split the text by "Output:" and take the second part
    if "Output:" in generated_text:
        return generated_text.split("Output:")[1].strip()
    return generated_text  # Fallback if "Output:" is not found

# evaluate the cleaned generated text

In [ ]:
cleaned_low_entropy_text = clean_generated_text(low_entropy_text)
cleaned_high_entropy_text = clean_generated_text(high_entropy_text)

print("Cleaned Low Entropy Text:")
print(cleaned_low_entropy_text)

print("Cleaned High Entropy Text:")
print(cleaned_high_entropy_text)

In [ ]:
print("Distinct-2 (Low Entropy):", distinct_n(cleaned_low_entropy_text, n=2))
print("Distinct-2 (High Entropy):", distinct_n(cleaned_high_entropy_text, n=2))

print("Repetition Rate (Low Entropy):", repetition_rate(cleaned_low_entropy_text, n=2))
print("Repetition Rate (High Entropy):", repetition_rate(cleaned_high_entropy_text, n=2))

print("Perplexity (Low Entropy):", calculate_perplexity(cleaned_low_entropy_text, model, tokenizer))
print("Perplexity (High Entropy):", calculate_perplexity(cleaned_high_entropy_text, model, tokenizer))